In [1]:
import pandas as pd
import numpy as np
import csv
import os
import matplotlib.pyplot as plt
import cufflinks as cf
import plotly
import plotly.offline as py
import plotly.graph_objs as go
from plotly.tools import FigureFactory as FF
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import numpy as np
import datetime as dt
from plotly.offline import init_notebook_mode, iplot, plot, download_plotlyjs
%matplotlib inline
init_notebook_mode(connected=True)
import cufflinks as cf
cf.go_offline()
import pandas_profiling

In [21]:
spoose = pd.read_csv('snp_final.csv')

In [3]:
ism_manf = pd.read_excel('ism_manf_new.xlsx', header=5)

In [4]:
def clean_function_2(market, time):
    market.dropna(inplace=True)
    #drop original nulls
    market.reset_index(inplace=True)
    #reset index for below
    new_dates = []
    for i in range(0,market.shape[0]):
        new_dates.append(str(market.ECO_RELEASE_DT[i]))
        
    market['new_dates'] = new_dates
    #setting new dates to a format mergeble
    market['new_dates'] = pd.to_datetime(market.new_dates)
    #formatting new dates
    market['Time'] = time
    #make sure to add time in hh:mm:ss format
    market.set_index('new_dates', inplace=True)
    #setting dates to index
    market.Time = pd.to_timedelta(market.Time)
    #getting the right format for merge
    market.index = market.index + market.Time
    #right format for merge
    market.drop(columns=['Time'], inplace=True)
    #removing original time column
    market.index = pd.to_datetime(market.index)
    #finalising format for merge
    market.columns = [name.lower() for name in market.columns]
    #lower case all the columns
    market.bn_survey_median = round(market.bn_survey_median,0)
    #round survey median to nearest interger
    market['miss_on_exp'] = market.actual_release - market.bn_survey_median
    #miss feature engineering
    market['pc_miss'] = (market.miss_on_exp / market.bn_survey_median)*100
    #pct miss feature engineering
    market.drop(columns=['dates', 'index', 'eco_release_dt'], inplace=True)
    market.dropna(inplace=True)
    #Drop any potential nans
    
    return(market)
    

In [20]:
def market_analysis(market, eco):

    for i in range(0,61):
        market['min_vol{}'.format(i)] = (market.High.shift(-i)-market.Low.shift(-i))
        market['close{}'.format(i)] = (market.Close.shift(-i)- market.Open)
    
    market.set_index(market.Date, inplace=True)
    market.index = pd.to_datetime(market.index)
    market.drop(columns=['Symbol', 'Date', 'Volume'], inplace=True)
    
    analysis = pd.merge(eco, market, left_index=True, right_index=True)
    
    trial_vol = pd.merge(eco, market, left_index=True, right_index=True)
    
    min_vol_list = []
    for i in range(0,61):
        min_vol_list.append('min_vol{}'.format(i))
    
    trial_vol = trial_vol[min_vol_list]
    
    highest_vol_min = []
    highest_vol_value = []
    for i in range(0, trial_vol.shape[0]):
        highest_vol_min.append(trial_vol.iloc[i].sort_values(ascending=False).index[0])
        highest_vol_value.append(trial_vol.iloc[i].sort_values(ascending=False).values[0])
    
    trial_close = pd.merge(eco, market, left_index=True, right_index=True)
    
    close_list = []
    for i in range(0,61):
        close_list.append('close{}'.format(i))

    
    trial_close = trial_close[close_list]
    
    #bottom_out = []
    #for i in range(0, trial_close.shape[0]):
        #if trial_close.T[trial_close.T[str(trial_close.index[i])]==0].shape[0]==0:
            #bottom_out.append('na')
        #elif trial_close.T[trial_close.T[str(trial_close.index[i])]==0].shape[0]==1:
            #bottom_out.append(trial_close.T[trial_close.T[str(trial_close.index[i])]==0].index[0])
        #elif trial_close.T[trial_close.T[str(trial_close.index[i])]==0].index[0]=='close0':
                #bottom_out.append(trial_close.T[trial_close.T[str(trial_close.index[i])]==0].index[1])
        #else: 
            #bottom_out.append(trial_close.T[trial_close.T[str(trial_close.index[i])]==0].index[0])
        
        
    max_out_60 = []
    max_out_value = []
    min_out_60 = []
    min_out_value = []
    for i in range(0, trial_close.shape[0]):
        max_out_60.append(trial_close.iloc[i][trial_close.iloc[i]==trial_close.iloc[i].max()].index[0])
        min_out_60.append(trial_close.iloc[i][trial_close.iloc[i]==trial_close.iloc[i].min()].index[0])
        max_out_value.append(trial_close.iloc[i][trial_close.iloc[i]==trial_close.iloc[i].max()].values[0])
        min_out_value.append(trial_close.iloc[i][trial_close.iloc[i]==trial_close.iloc[i].min()].values[0])
    
    


    trial_final = analysis[['actual_release', 'bn_survey_median', 'miss_on_exp', 'pc_miss', 'close0', 'close5']]
    
    trial_final['max_out_60'] = max_out_60
    trial_final['min_out_60'] = min_out_60
    trial_final['max_out_value'] = max_out_value
    trial_final['min_out_value'] = min_out_value
    trial_final['highest_vol_min'] = highest_vol_min
    trial_final['highest_vol_value'] = highest_vol_value
    #trial_final['level_out'] = bottom_out
    
    trial_final['vol_std'] = trial_vol.T.describe().iloc[2]
    
    moves_5m_low = trial_final.sort_values('close5').head(10)
    moves_5m_high = trial_final.sort_values('close5', ascending=False).head(10)
    price_vol30_std = trial_final.sort_values('vol_std', ascending=False).head(10)
    price_vol30_high = trial_final.sort_values('highest_vol_value', ascending=False).head(10)
    moves_0m_low = trial_final.sort_values('close0').head(10)
    moves_0m_high = trial_final.sort_values('close0', ascending=False).head(10)
    high_miss = trial_final.sort_values('miss_on_exp', ascending=False).head(10)
    high_beat = trial_final.sort_values('miss_on_exp').head(10)
    above_below = trial_final[(trial_final.bn_survey_median<=49) & (trial_final.bn_survey_median>=48)]
    
    highest_vol_min = []
    max_out_60_min = []
    min_out_60_min = []

    for i in range(0, trial_final.shape[0]):
        highest_vol_min.append(trial_final.highest_vol_min[i][7:])
        max_out_60_min.append(trial_final.max_out_60[i][5:])
        min_out_60_min.append(trial_final.min_out_60[i][5:])

    trial_final.highest_vol_min = highest_vol_min
    trial_final.max_out_60 = max_out_60_min
    trial_final.min_out_60 = min_out_60_min 
        
    highest_vol_min = []
    max_out_60_min = []
    min_out_60_min = []
    for i in range(0, trial_final.shape[0]):
        highest_vol_min.append(int(trial_final.highest_vol_min[i]))
        max_out_60_min.append(int(trial_final.max_out_60[i]))
        min_out_60_min.append(int(trial_final.min_out_60[i]))
        
    trial_final.highest_vol_min = highest_vol_min
    trial_final.max_out_60 = max_out_60_min
    trial_final.min_out_60 = min_out_60_min
    
    correlations = trial_final[['miss_on_exp','close0', 'close5', 'highest_vol_min',\
                 'highest_vol_value', 'max_out_60', 'min_out_60',\
             'max_out_value', 'min_out_value', 'vol_std']].corr()
    
    fig = trial_close.iloc[-9:].T.iplot()
    
    trial_final_plot = trial_final.copy()
    
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=trial_final_plot.index, y=trial_final_plot['close0'],
        name='close0',
        mode='lines',
        text= trial_final['highest_vol_min']

    ))

    fig.add_trace(go.Scatter(
        x=trial_final_plot.index, y=trial_final_plot['miss_on_exp'],
        name='miss_on_exp',
        marker_color='rgba(255, 182, 193, .9)',
        text= trial_final['highest_vol_value']
    ))

    # Set options common to all traces with fig.update_traces
    fig.update_traces(mode='lines', marker_line_width=2, marker_size=10)


    fig.show()

    
    df1 = pd.DataFrame(correlations)
    df2 = above_below
    df3 = moves_0m_low
    df4 = moves_0m_high
    df5 = high_miss
    df6 = high_beat
    
    return display(df1), display(df2), display(df3), display(df4), display(df5), display(df6)

In [6]:
clean_function_2(ism_manf, '15:00:00')

,actual_release,bn_survey_median,miss_on_exp,pc_miss
1996-11-01 15:00:00,50.2,51.0,-0.8,-1.568627
1996-12-02 15:00:00,52.7,51.0,1.7,3.333333
1997-01-02 15:00:00,55.2,52.0,3.2,6.153846
1997-02-03 15:00:00,52.0,53.0,-1.0,-1.886792
1997-03-03 15:00:00,53.1,52.0,1.1,2.115385
1997-04-01 15:00:00,55.0,53.0,2.0,3.773585
1997-05-01 15:00:00,54.2,54.0,0.2,0.370370
1997-06-02 15:00:00,57.1,54.0,3.1,5.740741
1997-07-01 15:00:00,55.7,56.0,-0.3,-0.535714
1997-08-01 15:00:00,58.6,56.0,2.6,4.642857


In [22]:
market_analysis(spoose, ism_manf)

,miss_on_exp,close0,close5,highest_vol_min,highest_vol_value,max_out_60,min_out_60,max_out_value,min_out_value,vol_std
miss_on_exp,1.000000,0.114258,0.653792,-0.010378,-0.141421,0.283492,-0.331566,0.395054,0.616115,-0.190939
close0,0.114258,1.000000,0.277379,0.178251,-0.032963,0.079635,0.014767,0.198615,0.224557,-0.031696
close5,0.653792,0.277379,1.000000,0.037676,-0.025742,0.169475,-0.238332,0.592795,0.725347,-0.100857
highest_vol_min,-0.010378,0.178251,0.037676,1.000000,-0.124472,-0.028116,0.200205,-0.053200,-0.007714,-0.091607
highest_vol_value,-0.141421,-0.032963,-0.025742,-0.124472,1.000000,-0.240173,-0.036146,0.354558,-0.370583,0.969151
max_out_60,0.283492,0.079635,0.169475,-0.028116,-0.240173,1.000000,-0.550653,0.440244,0.479370,-0.192443
min_out_60,-0.331566,0.014767,-0.238332,0.200205,-0.036146,-0.550653,1.000000,-0.478890,-0.522347,-0.039610
max_out_value,0.395054,0.198615,0.592795,-0.053200,0.354558,0.440244,-0.478890,1.000000,0.476930,0.376513
min_out_value,0.616115,0.224557,0.725347,-0.007714,-0.370583,0.479370,-0.522347,0.476930,1.000000,-0.426831
vol_std,-0.190939,-0.031696,-0.100857,-0.091607,0.969151,-0.192443,-0.039610,0.376513,-0.426831,1.000000


,actual_release,bn_survey_median,miss_on_exp,pc_miss,close0,close5,max_out_60,min_out_60,max_out_value,min_out_value,highest_vol_min,highest_vol_value,vol_std
2011-09-01 15:00:00,50.6,48.0,2.6,5.416667,0.50,11.75,close4,close0,15.25,0.50,min_vol1,15.25,1.827621
2016-01-04 15:00:00,48.2,49.0,-0.8,-1.632653,-0.50,-7.75,close21,close59,0.25,-11.25,min_vol3,5.25,0.857425
2016-02-01 15:00:00,48.2,48.0,0.2,0.416667,-1.75,-6.00,close58,close4,5.75,-6.00,min_vol12,3.75,0.636439
2016-03-01 15:00:00,49.5,48.0,1.5,3.125000,-0.50,3.00,close36,close0,11.75,-0.50,min_vol16,4.25,0.727391


,actual_release,bn_survey_median,miss_on_exp,pc_miss,close0,close5,max_out_60,min_out_60,max_out_value,min_out_value,highest_vol_min,highest_vol_value,vol_std
2016-02-01 15:00:00,48.2,48.0,0.2,0.416667,-1.75,-6.00,close58,close4,5.75,-6.00,min_vol12,3.75,0.636439
2013-06-03 15:00:00,49.0,51.0,-2.0,-3.921569,-1.75,1.25,close22,close43,6.00,-5.50,min_vol4,5.50,0.725323
2015-10-01 15:00:00,50.2,51.0,-0.8,-1.568627,-1.50,-2.75,close19,close2,7.25,-5.00,min_vol2,4.25,0.698359
2015-09-01 15:00:00,51.1,52.0,-0.9,-1.730769,-1.50,-7.75,close37,close6,6.25,-9.75,min_vol1,5.25,0.844764
2015-05-01 15:00:00,51.5,52.0,-0.5,-0.961538,-1.25,1.00,close21,close42,4.00,-2.50,min_vol19,2.50,0.399111
2018-04-02 15:00:00,59.3,60.0,-0.7,-1.166667,-1.25,-6.50,close0,close60,-1.25,-25.00,min_vol3,5.25,0.901501
2018-06-01 15:00:00,58.7,58.0,0.7,1.206897,-1.25,0.75,close23,close17,5.50,-6.00,min_vol23,5.25,0.810240
2016-05-02 15:00:00,50.8,51.0,-0.2,-0.392157,-1.25,-1.00,close17,close1,5.25,-2.75,min_vol1,3.75,0.653229
2013-08-01 15:00:00,55.4,52.0,3.4,6.538462,-1.25,1.50,close26,close0,3.25,-1.25,min_vol1,2.00,0.347218
2019-04-01 15:00:00,55.3,54.0,1.3,2.407407,-1.25,4.75,close6,close0,5.00,-1.25,min_vol1,4.75,0.666325


,actual_release,bn_survey_median,miss_on_exp,pc_miss,close0,close5,max_out_60,min_out_60,max_out_value,min_out_value,highest_vol_min,highest_vol_value,vol_std
2018-07-02 15:00:00,60.2,58.0,2.2,3.793103,2.50,6.25,close43,close15,8.75,-1.25,min_vol1,3.50,0.603609
2018-12-03 15:00:00,59.3,58.0,1.3,2.241379,2.00,3.75,close27,close60,9.25,-5.50,min_vol52,6.75,0.865887
2018-11-01 15:00:00,57.7,59.0,-1.3,-2.203390,1.75,-1.50,close24,close4,5.50,-2.75,min_vol51,4.25,0.842274
2012-06-01 15:00:00,53.5,54.0,-0.5,-0.925926,1.75,6.00,close5,close43,6.00,-1.50,min_vol1,3.50,0.622961
2019-08-01 15:00:00,51.2,52.0,-0.8,-1.538462,1.50,-0.75,close59,close4,17.50,-1.50,min_vol8,3.75,0.695639
2019-08-01 15:00:00,51.2,52.0,-0.8,-1.538462,1.50,-0.75,close59,close4,17.50,-1.50,min_vol8,3.75,0.695639
2015-04-01 15:00:00,51.5,52.0,-0.5,-0.961538,1.25,2.25,close51,close1,11.75,0.50,min_vol51,4.00,0.654273
2016-04-01 15:00:00,51.8,51.0,0.8,1.568627,1.00,2.50,close60,close0,14.75,1.00,min_vol6,4.25,0.643112
2015-11-02 15:00:00,50.1,50.0,0.1,0.200000,1.00,2.50,close51,close16,4.00,0.00,min_vol2,2.00,0.382373
2012-01-03 15:00:00,53.9,54.0,-0.1,-0.185185,0.75,3.25,close22,close0,5.00,0.75,min_vol1,2.50,0.343708


,actual_release,bn_survey_median,miss_on_exp,pc_miss,close0,close5,max_out_60,min_out_60,max_out_value,min_out_value,highest_vol_min,highest_vol_value,vol_std
2013-08-01 15:00:00,55.4,52.0,3.4,6.538462,-1.25,1.50,close26,close0,3.25,-1.25,min_vol1,2.00,0.347218
2018-09-04 15:00:00,61.3,58.0,3.3,5.689655,0.00,2.75,close56,close0,10.00,0.00,min_vol1,4.25,0.633804
2011-07-01 15:00:00,55.3,52.0,3.3,6.346154,0.25,7.25,close36,close0,11.25,0.25,min_vol1,5.25,0.717606
2014-11-03 15:00:00,59.0,56.0,3.0,5.357143,0.00,-0.25,close3,close23,2.50,-5.00,min_vol1,4.75,0.617178
2017-10-02 15:00:00,60.8,58.0,2.8,4.827586,0.75,0.75,close17,close1,3.50,0.50,min_vol8,1.75,0.309900
2017-09-01 15:00:00,58.8,56.0,2.8,5.000000,0.25,-0.50,close59,close8,3.25,-1.25,min_vol1,1.75,0.368778
2017-07-03 15:00:00,57.8,55.0,2.8,5.090909,-0.25,0.25,close14,close26,3.00,-1.25,min_vol26,2.00,0.318970
2011-02-01 15:00:00,60.8,58.0,2.8,4.827586,0.00,0.25,close52,close10,4.25,-1.25,min_vol1,2.75,0.392554
2011-09-01 15:00:00,50.6,48.0,2.6,5.416667,0.50,11.75,close4,close0,15.25,0.50,min_vol1,15.25,1.827621
2019-02-01 15:00:00,56.6,54.0,2.6,4.814815,-0.75,8.25,close26,close0,15.75,-0.75,min_vol1,7.25,0.975240


,actual_release,bn_survey_median,miss_on_exp,pc_miss,close0,close5,max_out_60,min_out_60,max_out_value,min_out_value,highest_vol_min,highest_vol_value,vol_std
2014-02-03 15:00:00,51.3,56.0,-4.7,-8.392857,-0.75,-6.25,close0,close20,-0.75,-14.75,min_vol1,6.75,0.888470
2019-01-03 15:00:00,54.1,58.0,-3.9,-6.724138,0.00,-11.00,close0,close39,0.00,-30.25,min_vol1,9.00,1.218006
2011-06-01 15:00:00,53.5,57.0,-3.5,-6.140351,-0.50,-2.75,close0,close51,-0.50,-10.75,min_vol1,2.25,0.386504
2011-08-01 15:00:00,50.9,54.0,-3.1,-5.740741,-0.25,-14.50,close0,close5,-0.25,-14.50,min_vol1,7.00,1.101306
2013-04-01 15:00:00,51.3,54.0,-2.7,-5.000000,-0.25,-3.25,close0,close60,-0.25,-7.75,min_vol1,5.00,0.636627
2016-09-01 15:00:00,49.4,52.0,-2.6,-5.000000,0.25,1.25,close11,close34,2.00,-10.25,min_vol1,3.50,0.673031
2015-01-02 15:00:00,55.5,58.0,-2.5,-4.310345,0.50,-1.50,close0,close44,0.50,-15.00,min_vol4,3.75,0.600290
2012-07-02 15:00:00,49.7,52.0,-2.3,-4.423077,-0.50,-7.00,close0,close9,-0.50,-8.25,min_vol1,5.50,0.776556
2019-05-01 15:00:00,52.8,55.0,-2.2,-4.000000,0.25,-1.00,close12,close31,1.25,-2.75,min_vol1,4.25,0.596037
2019-10-01 15:00:00,47.8,50.0,-2.2,-4.400000,-0.25,-13.50,close0,close60,-0.25,-28.25,min_vol1,10.50,1.485966


(None, None, None, None, None, None)

In [8]:
def chart(Date, market, release_time, end_time):
    test = market.loc[Date]
    test = test[(test.index.hour > release_time -1) & (test.index.hour <= release_time) & (test.index.minute < end_time)]
    
    fig = go.Figure(data=[go.Candlestick(x=test.index, open=test.Open, high=test.High,
                low=test.Low, close=test.Close)])
    
    
    return fig.show()

In [9]:
chart('2011-09-01', spoose, 15, 60)

In [12]:
4>=5

False